In [1]:
import numpy as np
import pandas as pd
import os
import win32com.client as win32

In [2]:
# https://www.youtube.com/watch?v=ZS4d3JvbQHQ

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.to_excel('titanictest.xlsx', index = False)

In [5]:
# Abrir el archivo de excel con la ruta completa
ruta = r'C:\Users\José Dos Reis\Documents\bootcamp_clases\titanictest.xlsx'
excel = win32.Dispatch('Excel.Application')
excel.Visible = False
wb = excel.Workbooks.Open(ruta)

In [6]:
#Seleccionar la hoja con los datos
ws_data = wb.Sheets('Sheet1')

#Agregar una nueva hoja para la tabla dinámica
ws_pivot = wb.Sheets.Add()
ws_pivot.Name = 'PivotTable'

In [7]:
# Rango de datos
start_cell = ws_data.Cells(1, 1)
end_cell = ws_data.Cells(ws_data.UsedRange.Rows.Count, ws_data.UsedRange.Columns.Count)
data_range = ws_data.Range(start_cell, end_cell)

In [8]:
# Crear el cache de la tabla dinámica
# pivot_cache = wb.PivotTableWizard(SourceType = 1, SourceData = data_range)

pivot_cache = wb.PivotCaches().Create(SourceType=1, SourceData=data_range)

In [9]:
# Crear la tabla dinámica
pivot_table = pivot_cache.CreatePivotTable(TableDestination=ws_pivot.Cells(3, 3), TableName='PivotTable1')

In [10]:
# Orientation = 1 es para las filas https://www.youtube.com/watch?v=ZS4d3JvbQHQ
pivot_table.PivotFields('Pclass').Orientation = 1
pivot_table.PivotFields('Pclass').Position = 1

In [11]:
pivot_table.PivotFields('Sex').Orientation = 1

In [12]:
# Esto es para hacerlo en formato tabular
pivot_table.RowAxisLayout(1)

In [13]:
# Esto quita los totales de columna
pivot_table.ColumnGrand = False

In [14]:
#Esto quita los subtotales de la tabla dinámica el 12 se debe a que hay 12 tipos de subtotales segun la documentacion
#https://learn.microsoft.com/en-us/office/vba/api/excel.pivotfield.subtotals
pivot_table.PivotFields('Pclass').Subtotals = [False for _ in range(12)]

In [15]:
pivot_table.PivotFields('Fare').Orientation = 4 #Orientation 4 es para meterlo como valores
#pivot_table.PivotFields('Fare').Function = -4139

com_error: (-2147352567, 'Ocurrió una excepción.', (0, 'Microsoft Excel', 'No se puede asignar la propiedad Function de la clase PivotField.', 'xlmain11.chm', 0, -2146827284), None)

In [16]:
# Aquí agrego una variable a el field de data y luego lo parseo a su función según la documentacion https://learn.microsoft.com/en-us/office/vba/api/excel.xlconsolidationfunction

values_field = pivot_table.AddDataField(pivot_table.PivotFields('Fare'))
values_field.Function = -4106

In [17]:
wb.Save()
wb.Close(SaveChanges=True)
excel.Quit()